In [2]:
from gensim.models import KeyedVectors
import time

/root/miniconda3/envs/torch16/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
t0 = time.time()
word2vec = KeyedVectors.load_word2vec_format('/root/pretrain-models/word2vec/Tencent_AILab_ChineseEmbedding.txt', binary=False)
print(time.time() - t0)

797.5861277580261


In [4]:
t0 = time.time()
word2vec = KeyedVectors.load('/root/pretrain-models/word2vec/Tencent_AILab_ChineseEmbedding.bin', mmap='r')
print(time.time() - t0)

6.427666187286377


In [5]:
title_exclude_list = [
    "交接",
    "串联",
    "新串联",
    "稍后",
    "提要",
    "开始语",
    "结束语",
    "（连放",
    "【连放"
]


def check_file(file):
    if os.path.exists(file):
        os.remove(file)
    fp = open(file, "w", encoding="utf-8")
    fp.close()


def parse_html(txt):
    # 解析txt
    xml_str = unescape(txt)  # html -> html str
    html = etree.HTML(xml_str)
    result = html.xpath("//p//text()")

    # 去掉[正文]末尾的空格
    result = [res.strip() for res in result]

    # 提取正文和导语之后的内容
    text = result[result.index("[正文]"):result.index("[导语]")]
    introduction = result[result.index("[导语]"):result.index("[编后]")]
    mark_txt = ""
    for introduction_str in introduction:
        if len(introduction_str) > 10:
            mark_txt = introduction_str
            break
    if mark_txt == "":
        for text_str in text:
            if len(text_str) > 10 and text_str[0] not in ("【", "[", "") and not text_str.startswith("素材"):
                mark_txt = text_str
                break
    last_str = ""
    for text_str in text[::-1]:
        if len(text_str) > 10 and text_str[0] not in ("【", "[", "") and not text_str.startswith("东方卫视"):
            last_str += text_str
            logger.info(last_str)
            break
    if last_str == "":
        for introduction_str in introduction[::-1]:
            if len(introduction_str) > 10:
                last_str = introduction_str
                break
    txt_str = ""
    for text_enum in text:
           txt_str = txt_str + text_enum
    intro_str = ""
    for intro_enum in introduction:
           intro_str = intro_str + intro_enum

    for i in punctuation:
        txt_str = txt_str.replace(i, "")
    for i in string.punctuation:
        txt_str = txt_str.replace(i, "")
    txt_str = txt_str.replace(" ", "")
    txt_str = cn2an.transform(txt_str, "an2cn")

    for i in punctuation:
        intro_str = intro_str.replace(i, "")
    for i in string.punctuation:
        intro_str = intro_str.replace(i, "")
    intro_str = intro_str.replace(" ", "")
    intro_str = cn2an.transform(intro_str, "an2cn")
    return txt_str, intro_str, mark_txt, last_str


def analysis_series_bill(xml_path):
    # 解析xml
    root = ET.parse(xml_path).getroot()
    # 遍历txt
    data = []
    for node in root.iter("text"):
        # num
        num = node.find("num").text
        title = node.find("title").text
        txt = node.find("txt").text
        words = node.find("words").text

        # 解析txt
        text, introduction, mark_txt, last_txt = parse_html(txt)

        audiotime = node.find("audiotime").text
        runtime = node.find("runtime").text
        totaltime = node.find("totaltime").text

        # for exclude_title in title_exclude_list:
        #     if re.match(exclude_title, title, flags=0):
        #         mark_txt = ""

        line = {"num": num, "title": title, "txt": text, "introduction": introduction, "audio_time": audiotime,
                "runtime": runtime, "total_time": totaltime, "mark_txt": mark_txt, "last_txt": last_txt, "words": words}
        data.append(line)

    return data

In [6]:
analysis_series_bill('../data/news/20210620.xml')

NameError: name 'ET' is not defined